##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")


Volumes vs DBFS/FileStore → DBFS/FileStore is a legacy workspace-scoped file system mainly for notebooks and small files, while Volumes (Unity Catalog) are governed storage objects that provide enterprise-grade access control, lineage, and compliance for non-tabular data.

Why production teams prefer Volumes → Because Volumes integrate with Unity Catalog, they enforce fine-grained governance, auditing, and regulatory compliance, making them suitable for sensitive or regulated production data

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/default/default_volume/customer.txt","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/"))

dbutils.fs.cp("/Volumes/telecom_catalog_assign/default/default_volume/usage.txt","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"))

dbutils.fs.cp("/Volumes/telecom_catalog_assign/default/default_volume/region1","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/region1.csv")
dbutils.fs.cp("/Volumes/telecom_catalog_assign/default/default_volume/region2","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/region2.csv")
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/"))
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"))

df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/region1.csv",header=True,inferSchema=True,sep="|")
df1.show()


##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",header=True,inferSchema=True,sep="|",pathGlobFilter="*.csv",recursiveFileLookup=True)
df1.show()

df2 = spark.read.csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/region1.csv","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/region2.csv"],header=True,inferSchema=True,sep="|")
df2.show()

df3 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/",header=True,inferSchema=True,sep="|",pathGlobFilter="region*",recursiveFileLookup=True)
df3.show()

df4 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",header=True,inferSchema=True,sep="|",recursiveFileLookup=True)
df4.show()



##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
df1 = spark.read.option("sep",",").option("header",False).option("inferSchema",False).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df1.printSchema()
df1.show(5)

df2 = spark.read.options(sep=",",header=False,inferSchema=True).format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df2.printSchema()
df2.show(5)

##to do
# df2 = spark.read.options(sep="\t").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
# df2.show(5)


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
df2 = df2.toDF("id", "name", "age", "city", "plan")
df2.show(5)

schema1 = "id int, name string, age int, city string, plan string"
df3 = 

## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

###15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
1. Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format
5. Read any one of the above delta table in a dataframe and write it to another table

##16. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables


# Healthcare Data Utilities Usecase2

## Objective
This notebook demonstrates how to design Databricks notebook using Markdown
and how to work with Databricks utilities such as dbutils.fs, dbutils.widgets,
and dbutils.notebook using Volumes.


## Project Workflow
1. Create folder structure using Volumes
2. Create sample healthcare data
3. Perform file operations using dbutils.fs
4. Parameterize execution using widgets
5. Exit notebook with execution status

## Folder Structure

| Folder | Purpose |
|------|---------|
| raw | Incoming healthcare files |
| processed | Validated healthcare data |
| archive | Historical data |


## Learning Outcome
Our Aspirants will understand notebook design, parameterization, and fs, notebook, widgets using Databricks utilities.

1. Define Base Paths using python variable <br>
base_path = "/Volumes/workspace/default/volumewd36" <br>
Create raw_path, processed_path and archive_path as given below... <br>
raw_path = f"{base_path}/raw" <br>
processed_path = f"{base_path}/processed" <br>
archive_path = f"{base_path}/archive"

In [0]:
base_path = "/Volumes/we47/we47schema/we47volume"
raw_path = f"{base_path}/raw"
processed_path = f"{base_path}/processed"
archive_path = f"{base_path}/archive"

2. dbutils Usecase – Create Directories using the above path variables..

In [0]:
dbutils.fs.mkdirs(raw_path)
dbutils.fs.mkdirs(processed_path)
dbutils.fs.mkdirs(archive_path)

3. dbutils Usecase – Create Sample Healthcare File <br>
sample_data = """patient_id,patient_name,age,gender
1,John Doe,68,M
2,Jane Smith,54,F
"""

TODO: Write this file content into raw folder created earlier... using dbutils.fs.......

In [0]:
sanple_data =  "patient_id,patient_name,age,gender 1,John Doe,68,M 2,Jane Smith,54,F"
dbutils.fs.put(f"{raw_path}/sample_data.csv",sanple_data,overwrite=False)

4. dbutils Usecase - list the file created <br>
TODO: List all files available in raw folder using the dbutils command <br>
dbutils.fs......

In [0]:
dbutils.fs.ls(raw_path)

5. dbutils Usecase – Copy File (raw → processed)

In [0]:
dbutils.fs.cp(f"{raw_path}/sample_data.csv",f"{processed_path}/sample_data.csv")

6. dbutils widget usecase - Create dropdown and text widgets... <br>
TODO: Create a dropdown widget for environment (dev, qa, prod) using <br>
TODO: Create a text widget for owner name



In [0]:
#format btutils.widgets.****("widget_name","default_value","list of values","description")
dbutils.widgets.dropdown("environment_name","dev",["dev","qa","prod"],"please select an environment")
#format btutils.widgets.****("widget_name","default_value","description")
dbutils.widgets.text("owner_name","jani","please enter the owner name")

7. dbutils widget Usecase – Read Widget Values environment and owner and print in the screen

In [0]:
# get will return the string
env_name = dbutils.widgets.get("environment_name")
owner_name = dbutils.widgets.get("owner_name")
print(env_name)
print(owner_name)

#getAll returns the dictionary
all_widget_values = dbutils.widgets.getAll()
print(all_widget_values)

8. dbutils widget Usecase – Move the above processed File to Archive

In [0]:
dbutils.fs.mv(f"{processed_path}/sample_data.csv",f"{archive_path}/sample_data.csv")

9. dbutils notebook usecase - Run the notebook4 using the dbutils command
/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/databricks_workouts_2025/1_USECASES_NB_FUNDAMENTALS/4_child_nb_dataload

In [0]:
# we must give timeout when we run a notebook
dbutils.notebook.run("/Workspace/Git Folder/DataBricks_Practice/we47_local_notebooks/usecase_practice",60)

10. dbutils notebook usecase - exit this notebook 
TODO: Exit notebook with a success message
dbutils.notebook._____("Pipeline completed successfully")


In [0]:
dbutils.notebook.exit("pipeline completed successfully")